In [1]:
%matplotlib inline

In [2]:
import os
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import widgets, Layout
from IPython.display import display
from nipype.interfaces.nipy import SpaceTimeRealigner, ComputeMask
from nipype.interfaces.fsl import BET, MeanImage, ApplyMask, SwapDimensions
from nipype.interfaces.afni import SkullStrip, Automask
import seaborn as sns
from nipype.interfaces.ants import N4BiasFieldCorrection, Registration
from pca_utils import pca_denoising

/home/julia/workspace/conda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
in_dir = '/home/julia/projects/lc/from_barbara/'
out_dir = '/home/julia/projects/lc/from_barbara/'
slab_file = in_dir + 'BCLCmice3_19.nii.gz'
slab_data = nb.load(slab_file).get_data()
slab_aff = nb.load(slab_file).affine
slab_hdr = nb.load(slab_file).header

In [5]:
if not os.path.isdir(out_dir+'single_vols'):
    os.mkdir(out_dir+'single_vols')

In [7]:
for i in range(slab_data.shape[3]):
    nb.save(nb.Nifti1Image(slab_data[:,:,:,i], slab_aff, slab_hdr),
            out_dir + 'single_vols/slab_vol%s.nii.gz'%i)
    n4 = N4BiasFieldCorrection(input_image=out_dir + 'single_vols/slab_vol%s.nii.gz'%i, dimension=3,
                               n_iterations=[100,100,100,100], convergence_threshold=0.0,
                               output_image= out_dir + 'single_vols/slab_vol%s_corrected.nii.gz'%i)
    n4.run()

In [8]:
corrected_data = []
for i in range(slab_data.shape[3]):
    corrected_data.append(nb.load(out_dir + 'single_vols/slab_vol%s_corrected.nii.gz'%i).get_data())

nb.save(nb.Nifti1Image(np.average(corrected_data[:], axis=0), slab_aff, slab_hdr),
        out_dir + 'slab_avg_corrected.nii.gz')